In [1]:
import pandas as pd
import csv
import os
import numpy as np
import cv2
pd.options.mode.chained_assignment = None

In [2]:
def read_data_with_subdirectorys(data_path, extension):
    videos_path_list = []
    print("List of all directories in '% s':" % data_path)

    for path, subdirs, files in os.walk(data_path):
        for name in files:
            if(extension is not None):
                if name.endswith(extension):
                    videos_path_list.append(os.path.join(path, name))
            else:
                videos_path_list.append(os.path.join(path, name))

    return videos_path_list


def read_concat_xml(path):
    
    df_results = pd.DataFrame(columns=['frame', 'start', 'end', 'video_name'])
    
    lista_xmls = read_data_with_subdirectorys(path, '.xml')
    file = None
    for file in lista_xmls:
        print(file)
        df = pd.read_xml(file)
        df = df.reset_index() #para pegar o frame
        df['index'] = df['index'] +1
        df = df.rename(columns={'index':'frame', 'v':'aus'})
        df['video_name'] = file.split('/')[-1].split('.')[0]
        print(df.shape)
        #print(df_results.shape)
        
        df_results = pd.concat([df_results, df], ignore_index=True)

    return df_results

def is_au(string):
    txt = string
    x = txt.replace("+", "")
    return x.isdigit()

In [3]:
path = "../data/outputs/Videos-Tamires-Selecionados/predicts_squeezenet/"
df = read_concat_xml(path)
df = df.sort_values(by=['video_name', 'start'], ascending=(False, True))
df

List of all directories in '../data/outputs/Videos-Tamires-Selecionados/predicts_squeezenet/':
../data/outputs/Videos-Tamires-Selecionados/predicts_squeezenet/RostoIntensidade-04Quarta-Aniquilar.xml
(17, 5)
../data/outputs/Videos-Tamires-Selecionados/predicts_squeezenet/RostoIntensidade-04Quarta-Sortudo.xml
(13, 5)


,frame,start,end,video_name,aus
17,1,0.000,0.133,RostoIntensidade-04Quarta-Sortudo,0
18,2,0.133,0.233,RostoIntensidade-04Quarta-Sortudo,25+26+42
19,3,0.233,0.366,RostoIntensidade-04Quarta-Sortudo,25+26+42+51
20,4,0.366,0.565,RostoIntensidade-04Quarta-Sortudo,22+25+26+42+51
21,5,0.565,0.632,RostoIntensidade-04Quarta-Sortudo,22+25+26+43+51
22,6,0.632,0.732,RostoIntensidade-04Quarta-Sortudo,22+25+43+51
23,7,0.732,0.799,RostoIntensidade-04Quarta-Sortudo,23+43+51
24,8,0.799,0.832,RostoIntensidade-04Quarta-Sortudo,43+51
25,9,0.832,0.899,RostoIntensidade-04Quarta-Sortudo,13+43+51
26,10,0.899,1.198,RostoIntensidade-04Quarta-Sortudo,13+42+51


In [4]:
def define_frame_init_final(df3, fps_video):

    #fps_video = 29.808
    df3['frame_inicial'] = np.ceil((df3['start'])*fps_video).astype(int)
    #df3['frame_final'] = df3['frame_inicial'].shift(-1, fill_value=0)
    df3['frame_final'] = np.ceil((df3['end'])*fps_video).astype(int)
    df3['frame_final'] = df3['frame_final'].apply(lambda x: x-1 if x>0 else x) #diminundo 1 frame para evitar sobreposição

    #Eliminando a ocorrência com frase
    df3["au_valid"] = df3.apply(lambda row: is_au(row['aus']), axis=1) 
    df3 = df3.loc[df3.au_valid == True][['frame', 'start', 'end', 'video_name', 'aus', 'frame_inicial', 'frame_final']]
    
    return df3

def aus_distribution(df3):
    df_true = pd.DataFrame(columns = ['video_name', 'frame', 'aus'])
    frame_max = df3.frame_final.max()
    print(frame_max)

    for i in range(0, frame_max):
        
        # Filtrando as rows para preencher as unidades de ação para cada frame.
        # Selecionando o intervalo de frames que terão unidades de ação iguais/repetidas
       
        row = df3.loc[(df3.frame_inicial <= i+1) & (df3.frame_final >= i+1)]
        print(row.video_name)
        new_row = {
            "video_name": row.video_name.item(),
            "frame": i+1,
            "aus": row.aus.item(),
        }

        df_true = df_true.append(new_row, ignore_index=True)
    return df_true
    


In [6]:
df_results = pd.DataFrame(columns = ['video_name', 'frame', 'aus'])

for i in df.video_name.unique():
        extensao = '.avi'
        print(path+i+extensao)
        v_entry = cv2.VideoCapture(path+i+extensao, 0)
        #frames = int(v_entry.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = v_entry.get(cv2.CAP_PROP_FPS)
        df_frames = define_frame_init_final(df.loc[df.video_name == i], fps)
        df_aus = aus_distribution(df_frames)
        
        df_results = pd.concat([df_aus, df_results], ignore_index=True)

display(df_results)        
    

../data/outputs/Videos-Tamires-Selecionados/predicts_squeezenet/RostoIntensidade-04Quarta-Sortudo.avi
70
17    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
17    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
17    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
18    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
18    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
18    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
19    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
19    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
19    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
19    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
20    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
20    RostoIntensidade-04Quarta-Sortudo
Name: video_name, dtype: object
20    RostoIntensidade-04Quarta

Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object
16    RostoIntensidade-04Quarta-Aniquilar
Name: video_name, dtype: object


,video_name,frame,aus
0,RostoIntensidade-04Quarta-Aniquilar,1,0
1,RostoIntensidade-04Quarta-Aniquilar,2,0
2,RostoIntensidade-04Quarta-Aniquilar,3,0
3,RostoIntensidade-04Quarta-Aniquilar,4,4
4,RostoIntensidade-04Quarta-Aniquilar,5,4
...,...,...,...
126,RostoIntensidade-04Quarta-Sortudo,66,12+42+51
127,RostoIntensidade-04Quarta-Sortudo,67,12+42+51
128,RostoIntensidade-04Quarta-Sortudo,68,12+42+51
129,RostoIntensidade-04Quarta-Sortudo,69,12+42+51


In [7]:
df_results.video_name.value_counts()

RostoIntensidade-04Quarta-Sortudo      70
RostoIntensidade-04Quarta-Aniquilar    61
Name: video_name, dtype: int64

In [8]:
df_results.loc[df_results.video_name == 'RostoIntensidade-04Quarta-Sortudo'].head(30)

,video_name,frame,aus
61,RostoIntensidade-04Quarta-Sortudo,1,0
62,RostoIntensidade-04Quarta-Sortudo,2,0
63,RostoIntensidade-04Quarta-Sortudo,3,0
64,RostoIntensidade-04Quarta-Sortudo,4,25+26+42
65,RostoIntensidade-04Quarta-Sortudo,5,25+26+42
66,RostoIntensidade-04Quarta-Sortudo,6,25+26+42
67,RostoIntensidade-04Quarta-Sortudo,7,25+26+42+51
68,RostoIntensidade-04Quarta-Sortudo,8,25+26+42+51
69,RostoIntensidade-04Quarta-Sortudo,9,25+26+42+51
70,RostoIntensidade-04Quarta-Sortudo,10,25+26+42+51


In [9]:
df_results.loc[df_results.video_name == 'RostoIntensidade-04Quarta-Aniquilar'].head(30)

,video_name,frame,aus
0,RostoIntensidade-04Quarta-Aniquilar,1,0
1,RostoIntensidade-04Quarta-Aniquilar,2,0
2,RostoIntensidade-04Quarta-Aniquilar,3,0
3,RostoIntensidade-04Quarta-Aniquilar,4,4
4,RostoIntensidade-04Quarta-Aniquilar,5,4
5,RostoIntensidade-04Quarta-Aniquilar,6,4
6,RostoIntensidade-04Quarta-Aniquilar,7,4+10+25
7,RostoIntensidade-04Quarta-Aniquilar,8,4+9+10+25
8,RostoIntensidade-04Quarta-Aniquilar,9,4+9+10+25
9,RostoIntensidade-04Quarta-Aniquilar,10,4+9+10+25+26


In [10]:
df_results = df_results.sort_values(by=['video_name', 'frame'], ascending=(False, True))
df_results

,video_name,frame,aus
61,RostoIntensidade-04Quarta-Sortudo,1,0
62,RostoIntensidade-04Quarta-Sortudo,2,0
63,RostoIntensidade-04Quarta-Sortudo,3,0
64,RostoIntensidade-04Quarta-Sortudo,4,25+26+42
65,RostoIntensidade-04Quarta-Sortudo,5,25+26+42
...,...,...,...
56,RostoIntensidade-04Quarta-Aniquilar,57,0
57,RostoIntensidade-04Quarta-Aniquilar,58,0
58,RostoIntensidade-04Quarta-Aniquilar,59,0
59,RostoIntensidade-04Quarta-Aniquilar,60,0


In [11]:
df_results = df_results.rename(columns={"aus": "ground truth"})

In [12]:
df_results.to_csv("../data/outputs/Videos-Tamires-Selecionados/predicts_squeezenet/Videos-Tamires-Selecionados-ground-truth.csv", sep=';')